In [ ]:
import os

Training

In [ ]:
python generate_tfrecord.py -x [PATH_TO_IMAGES_FOLDER]/train -l [PATH_TO_ANNOTATIONS_FOLDER]/label_map.pbtxt -o [PATH_TO_ANNOTATIONS_FOLDER]/train.record
# stuff in square bracket = to be filled. assumes train and label_map.pbtxt to be within said folders within the square bracket

In [ ]:
python generate_tfrecord.py -x [PATH_TO_IMAGES_FOLDER]/test -l [PATH_TO_ANNOTATIONS_FOLDER]/label_map.pbtxt -o [PATH_TO_ANNOTATIONS_FOLDER]/test.record
# assumes test and label_map.pbtxt to be within said folders within the square bracket

In [ ]:
python model_main_tf2.py --model_dir=[PATH_TO_DOWNLOADED_MODELS_FILE]/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8 --pipeline_config_path=[PATH_TO_DOWNLOADED_MODELS_FILE]/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config

Evaluation

In [ ]:
python model_main_tf2.py --model_dir=[PATH_TO_DOWNLOADED_MODELS_FILE]/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8 --pipeline_config_path=[PATH_TO_DOWNLOADED_MODELS_FILE]/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config -checkpoint_dir=[PATH_TO_DOWNLOADED_MODELS_FILE]/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8

Define variables for detection later

In [2]:
CUSTOM_MODEL_NAME = 'SSD_proto02' 
EXPORTED_MODEL_NAME = CUSTOM_MODEL_NAME + '_exp'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'labelmap.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','workspace'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','image'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','trained_models',CUSTOM_MODEL_NAME,'checkpoint'), 
    'EXPORTED_CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','exported_models',EXPORTED_MODEL_NAME,'checkpoint'), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','exported_models',EXPORTED_MODEL_NAME), 
    'PROTOC_PATH':os.path.join('Protobuf','bin')
}

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','trained_models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'EXPORTED_PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','exported_models', EXPORTED_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        print(path ,"not found")

Running detection

In [6]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
import time

In [7]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# restore exported model
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['EXPORTED_CHECKPOINT_PATH'], 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

2021-10-06 16:09:57.967067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 16:09:57.998729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 16:09:57.999153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 16:09:57.999982: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Define variables for use later

In [8]:
#define image path and create index based on labelmap classes
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'honda_crv_4thGen (56).jpg')

Define variables for use later

In [10]:
#read the image
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

#expand the dimensions of the image so that it is usable by tensorflow
image_np_expanded = np.expand_dims(image_np, axis=0)
#convert numpy array to tensor object
input_tensor = tf.convert_to_tensor(image_np_expanded, dtype=tf.float32)

#run detection and record time taken
start = time.time()
detections = detect_fn(input_tensor)
end = time.time()
dur = end - start
print(dur)

#draws the bounding boxes based on object detections
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

0.03345370292663574


In [13]:
VIDEO_PATH = os.path.join(paths['IMAGE_PATH'], 'videos', 'perodua_axia.mp4')
OUTPUT_VIDEO_PATH = os.path.join(paths['IMAGE_PATH'], 'output', 'output_axia_SSD_proto01.mp4')
cap = cv2.VideoCapture(VIDEO_PATH)
print(cap.isOpened())
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH,cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width,frame_height))
count = 0
totdur = 0

while cap.isOpened(): 
    ret, frame = cap.read()
    
    if (not ret):
        break
    
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    start = time.time()
    detections = detect_fn(input_tensor)
    end = time.time()
    dur = end - start
    totdur += dur
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    out.write(image_np_with_detections)
    
    count += 1
    if count%30 == 0:
        print((count/30),"seconds of video processed.")
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
print("total inference duration in seconds: ", totdur)
print("fps", count/totdur)
cap.release()
out.release()
cv2.destroyAllWindows()

True


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


1.0 seconds of video processed.
2.0 seconds of video processed.
3.0 seconds of video processed.
4.0 seconds of video processed.
5.0 seconds of video processed.
6.0 seconds of video processed.
7.0 seconds of video processed.
8.0 seconds of video processed.
9.0 seconds of video processed.
10.0 seconds of video processed.
11.0 seconds of video processed.
12.0 seconds of video processed.
13.0 seconds of video processed.
14.0 seconds of video processed.
15.0 seconds of video processed.
16.0 seconds of video processed.
17.0 seconds of video processed.
18.0 seconds of video processed.
19.0 seconds of video processed.
20.0 seconds of video processed.
21.0 seconds of video processed.
22.0 seconds of video processed.
23.0 seconds of video processed.
24.0 seconds of video processed.
25.0 seconds of video processed.
26.0 seconds of video processed.
27.0 seconds of video processed.
28.0 seconds of video processed.
29.0 seconds of video processed.
30.0 seconds of video processed.
31.0 seconds of vid